<a href="https://colab.research.google.com/github/arbouria/Notas-Aprendizaje-y-Comportamiento-Adaptable-I/blob/main/Copia_de_Baum_evitaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importar las librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive_output, VBox, Layout

# Estilo para los gráficos
plt.style.use('seaborn-v0_8-whitegrid')

# --- Simulador 1: Extensión del Modelo de Error de Predicción a la Evitación ---

def simulate_avoidance_td(cs_us_interval, response_time):
    """
    Simula la dinámica del "valor de peligro" y el "error de alivio" en un solo ensayo de evitación.
    """
    if response_time > cs_us_interval:
        response_time = cs_us_interval

    time_steps = np.linspace(0, cs_us_interval, num=100)
    danger_value = np.zeros_like(time_steps)
    relief_error = np.zeros_like(time_steps)

    # El valor del estado (peligro) cae a medida que se acerca el shock
    # La tasa de caída depende de la urgencia (inversa del intervalo CS-US)
    max_danger = -1.0 # El valor más negativo que representa el shock inminente

    for i, t in enumerate(time_steps):
        if t < response_time:
            # Caída lineal hacia el peligro máximo
            danger_value[i] = max_danger * (t / cs_us_interval)
        else:
            # La respuesta resetea el peligro a cero
            danger_value[i] = 0.0

    # El error de predicción de "alivio" solo ocurre en el momento de la respuesta
    response_index = np.argmin(np.abs(time_steps - response_time))
    value_before_response = max_danger * (response_time / cs_us_interval)
    value_after_response = 0.0
    relief_error[response_index] = value_after_response - value_before_response

    # Crear los gráficos
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

    # Gráfico 1: Valor de Peligro
    ax1.plot(time_steps, danger_value, 'r-', label='Valor de Peligro (Negativo)')
    ax1.axvline(response_time, color='k', linestyle=':', label='Momento de la Respuesta')
    ax1.set_title(f'Dinámica del Peligro (Intervalo CS-US = {cs_us_interval}s)', fontsize=14)
    ax1.set_ylabel('Valor del Estado')
    ax1.set_ylim(-1.1, 0.1)
    ax1.legend()

    # Gráfico 2: Error de Predicción de Alivio
    ax2.stem(time_steps, relief_error, 'g', markerfmt='go', basefmt=" ")
    ax2.axvline(response_time, color='k', linestyle=':', label='Momento de la Respuesta')
    ax2.set_title(f'Señal de "Alivio" (Error de Predicción Positivo = {relief_error[response_index]:.2f})', fontsize=14)
    ax2.set_ylabel('Error de Predicción ($\delta$)')
    ax2.set_xlabel('Tiempo desde el inicio del CS (s)')
    ax2.set_ylim(0, 1.1)

    plt.tight_layout()
    plt.show()

# Widgets para el simulador de evitación TD
style = {'description_width': 'initial'}
avoidance_td_widgets = VBox([
    widgets.IntSlider(value=10, min=2, max=20, step=1, description='Intervalo CS-US (s):', style=style),
    widgets.IntSlider(value=5, min=1, max=20, step=1, description='Momento de la Respuesta (s):', style=style)
])

# --- Simulador 2: Extensión de la Teoría de la Información a la Evitación ---

def simulate_avoidance_info(ss_interval, rs_interval):
    """
    Calcula la informatividad de la respuesta de evitación.
    """
    if rs_interval <= 0 or ss_interval <=0:
        print("Los intervalos deben ser mayores que cero.")
        return

    # La informatividad de la RESPUESTA
    informativeness = ss_interval / rs_interval

    # Una forma de medir la "fuerza" del aprendizaje instrumental podría ser el logaritmo
    # de la informatividad, que corresponde a la Información Mutua.
    if informativeness <= 1:
        mutual_info = 0
        strength_desc = "Baja (la respuesta no es más informativa que el azar)"
    else:
        mutual_info = np.log2(informativeness) # en bits
        strength_desc = f"Alta ({mutual_info:.2f} bits)"

    print("--- Resultados de la Teoría de la Información (Evitación) ---")
    print(f"Intervalo Shock-Shock (S-S) | Peligro de fondo: {ss_interval} s")
    print(f"Intervalo Respuesta-Shock (R-S) | Seguridad comprada: {rs_interval} s")
    print("---------------------------------------------------------------")
    print(f"Informatividad de la RESPUESTA (S-S / R-S): {informativeness:.2f}")
    print(f"Fuerza de la Contingencia Instrumental: {strength_desc}")
    print("\n*Nota: La respuesta se aprende mejor cuando es muy informativa, es decir, cuando compra un largo período de seguridad (R-S alto) en un entorno donde los shocks son frecuentes (S-S bajo).")


# Widgets para el simulador de evitación por información
avoidance_info_widgets = VBox([
    widgets.IntSlider(value=10, min=2, max=100, step=1, description='Intervalo S-S (Peligro de fondo):', style=style),
    widgets.IntSlider(value=20, min=1, max=100, step=1, description='Intervalo R-S (Seguridad comprada):', style=style)
])

# --- Visualización de los Simuladores ---
tabs_avoidance = widgets.Tab()

# Simulador de Evitación TD/Kalman
td_out = interactive_output(simulate_avoidance_td, {'cs_us_interval': avoidance_td_widgets.children[0], 'response_time': avoidance_td_widgets.children[1]})
tab_td = VBox([widgets.HTML("<h2>Simulador 1: Evitación como Error de Predicción de 'Alivio'</h2>"), avoidance_td_widgets, td_out])

# Simulador de Evitación por Información
info_out_avoid = interactive_output(simulate_avoidance_info, {'ss_interval': avoidance_info_widgets.children[0], 'rs_interval': avoidance_info_widgets.children[1]})
tab_info_avoid = VBox([widgets.HTML("<h2>Simulador 2: Evitación como Información de la Respuesta</h2>"), avoidance_info_widgets, info_out_avoid])

tabs_avoidance.children = [tab_td, tab_info_avoid]
tabs_avoidance.set_title(0, 'Modelo Error de Predicción')
tabs_avoidance.set_title(1, 'Modelo Teoría de la Información')

display(tabs_avoidance)